<a href="https://colab.research.google.com/github/alex2844/padavan-builder/blob/main/build.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# @title Variables
import os
import re
import requests
import zipfile
from google.colab import files, drive

REPO = 'https://gitlab.com/hadzhioglu/padavan-ng.git' # @param {"type":"string","placeholder":" "}
os.environ['PADAVAN_REPO'] = REPO

BRANCH = 'master' # @param {"type":"string","placeholder":" "}
os.environ['PADAVAN_BRANCH'] = BRANCH

COMMIT = 'HEAD' # @param {"type":"string","placeholder":" "}
os.environ['PADAVAN_COMMIT'] = COMMIT

TOOLCHAIN_URL = 'https://gitlab.com/api/v4/projects/hadzhioglu%2Fpadavan-ng/packages/generic/toolchain/latest/toolchain.tzst' # @param {"type":"string","placeholder":" "}
os.environ['PADAVAN_TOOLCHAIN_URL'] = TOOLCHAIN_URL

CONFIG = 'padavan-ng/trunk/configs/templates/xiaomi/mi-4a_100m.config' # @param {"type":"string","placeholder":" "}
os.environ['PADAVAN_CONFIG'] = CONFIG

THEMES_REPO = 'https://gitlab.com/hadzhioglu/padavan-themes.git' # @param ["https://gitlab.com/hadzhioglu/padavan-themes.git", "https://gitlab.com/hadzhioglu/padavan-themes-lite.git"] {allow-input: true}
os.environ['PADAVAN_THEMES_REPO'] = THEMES_REPO

THEMES_BRANCH = 'main' # @param {"type":"string","placeholder":" "}
os.environ['PADAVAN_THEMES_BRANCH'] = THEMES_BRANCH

THEMES = '' # @param {"type":"string","placeholder":" "}
os.environ['PADAVAN_THEMES'] = ' '.join([option.strip() for option in THEMES.split(',')])

ZIP = 'download' # @param ["download", "save"]

if ZIP == 'save' or "drive/MyDrive/" in CONFIG:
  if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

In [ ]:
# @title Config {"form-width":"150px"}
# @markdown [![Templates](https://img.shields.io/badge/Templates-config-red.svg)](https://gitlab.com/hadzhioglu/padavan-ng/-/tree/master/trunk/configs/templates)
%%writefile build.config


Writing build.config


In [12]:
# @title Download sources
%%shell
git config --global --add safe.directory '*'
if [[ -n "$PADAVAN_COMMIT" ]] && [ "$PADAVAN_COMMIT" != "HEAD" ]; then
  git clone -b "$PADAVAN_BRANCH" "$PADAVAN_REPO"
  git -C padavan-ng checkout "$PADAVAN_COMMIT"
else
  git clone --depth 1 -b "$PADAVAN_BRANCH" "$PADAVAN_REPO"
fi
if [[ -n $PADAVAN_THEMES ]]; then
  git clone --depth 1 -b "$PADAVAN_THEMES_BRANCH" "$PADAVAN_THEMES_REPO" themes
  cp -r themes/common-theme themes/jquery.js padavan-ng/trunk/user/www/n56u_ribbon_fixed
  for theme in ${PADAVAN_THEMES[@]}; do
    echo "Installing $theme theme"
    cp -r "themes/$theme-theme" padavan-ng/trunk/user/www/n56u_ribbon_fixed
  done
fi

In [ ]:
# @title Dockerfile to shell script
%%writefile dkr2sh.sed
#!/bin/sed -f
# Convert Dockerfile to shell script.
# Copyright 2018, Development Gateway, GPLv3+
# Copyright 2022, Dennis Schneidermann, GPLv3+ - modifications noted by "dss"

# remove pure comment lines to support multi-line RUN statements - added by dss 20220223
/^[[:space:]]*#.*/d;

# comment out other instructions
/^[[:space:]]*\<\(FROM\|CMD\|LABEL\|MAINTAINER\|EXPOSE\|ENTRYPOINT\|VOLUME\|USER\|ONBUILD\|STOPSIGNAL\|HEALTHCHECK\|SHELL\)\>/I s/^[[:space:]]*/# /;

# ENV and ARG instructions become export commands - modified to support ARG by dss 20220223
s/^[[:space:]]*\(ENV\|ARG\)[[:space:]]\+\([^[:space:]]\+\)[[:space:]]\+\(.\+\)/export \2=\3/i;

# ENV and ARG instructions become export commands (alternate) - added by dss 20220223
s/^[[:space:]]*\(ENV\|ARG\)[[:space:]]\+\([^[:space:]]\+\)=\(.\+\)/export \2=\3/i;

# comment out any unparsable ENV or ARG instructions - added by dss 20220223
/^[[:space:]]*\<\(ENV\|ARG\)\>/I s/^[[:space:]]*/# (no value assigned) /;

# RUN get executed literally, including line wraps
s/^[[:space:]]*RUN[[:space:]]\+//i;

# COPY behaves differently on files and directories
# with a "--from" argument
s/^[[:space:]]*COPY\([[:space:]]\+--from=[^[:space:]]\+\)\(\([[:space:]]\+[^[:space:]]\+\)\+\)\([[:space:]]\+[^[:space:]]\+\)[[:space:]]*/for i in\2; do\n\ttest -d "$i" \&\& i="$i\/"\n\trsync -a "$i"\4 #\1\ndone/i
# without "--from" argument
s/^[[:space:]]*COPY\(\([[:space:]]\+[^[:space:]]\+\)\+\)\([[:space:]]\+[^[:space:]]\+\)[[:space:]]*/for i in\1; do\n\ttest -d "$i" \&\& i="$i\/"\n\trsync -a "$i"\3\ndone/i

# ADD
s/^[[:space:]]*ADD\(\([[:space:]]\+[^[:space:]]\+\)\+\)[[:space:]]\+\([^[:space:]]\+\)[[:space:]]*/if echo "\3" | grep -q '\/$'; then\n\tmkdir -p "\3";\nelse\n\tmkdir -p "$(dirname "\3")";\nfi\n\nfor i in\1; do\n\tif echo "$i" | grep -q '^\\(https\\?\\|ftp\\):\/\/'; then\n\t\tif echo "\3" | grep -q '\/$'; then\n\t\t\t(cd "\3" \&\& wget -q "$i");\n\t\telse\n\t\t\twget -q -O "\3" "$i";\n\t\tfi\n\telif test -d "$i"; then\n\t\trsync -a "$i\/" "\3";\n\telif tar -tf "$i" >\/dev\/null 2>\&1; then\n\t\tmkdir -p "\3" \&\& tar -C "\3" -xf "$i";\n\telse\n\t\trsync -a "$i" "\3";\n\tfi\ndone/i

# imitate WORKDIR behavior
s/^[[:space:]]*WORKDIR[[:space:]]\+\(.\+\)/mkdir -p \1 \&\& cd \1/i

Writing dkr2sh.sed


In [ ]:
# @title Dependencies
%%shell
chmod +x dkr2sh.sed
./dkr2sh.sed < padavan-ng/Dockerfile > Dockerfile.sh
chmod +x Dockerfile.sh
./Dockerfile.sh

In [ ]:
# @title Download toolchain
%%shell
wget -qO- "$PADAVAN_TOOLCHAIN_URL" | tar -C padavan-ng --zstd -xf -

In [ ]:
# @title Get config
PADAVAN_CONFIG = os.environ['PADAVAN_CONFIG']
if PADAVAN_CONFIG and PADAVAN_CONFIG != "build.config":
  regex = re.compile(r'^(https?|ftp|file)://', re.IGNORECASE)
  if re.match(regex, PADAVAN_CONFIG):
    response = requests.get(PADAVAN_CONFIG)
    response.raise_for_status()
    with open('build.config', 'wb') as f:
      f.write(response.content)
  else:
    ! cp "$PADAVAN_CONFIG" build.config

with open('build.config', 'r') as f:
  for line in f:
    line = line.strip()
    if line and '=' in line and not line.startswith('#'):
      key, value = line.split('=', 1)
      if value.startswith(('"', "'")) and value.endswith(('"', "'")):
        value = value[1:-1]
      os.environ[key] = value

In [ ]:
# @title Build firmware
%%shell
cp build.config padavan-ng/trunk/.config
cd padavan-ng/trunk
./build_firmware.sh

In [ ]:
# @title Prepare zip
%%bash
FW_FILE_NAME="$(find padavan-ng/trunk/images -type f -regextype posix-extended -iregex ".*\.(trx|bin)$" -printf "%T@\t%f\n" | sort -V | tail -1 | cut -f2)"
echo "FW_FILE_NAME=$FW_FILE_NAME" > build.fw
echo "FW_NAME=${FW_FILE_NAME%.*}" >> build.fw
mv "padavan-ng/trunk/images/$FW_FILE_NAME" .

In [ ]:
# @title Create zip
with open('build.fw', 'r') as f:
  for line in f:
    line = line.strip()
    if line and '=' in line:
      key, value = line.split('=', 1)
      os.environ[key] = value

zip = os.environ['CONFIG_VENDOR'] + '_' + os.environ['FW_NAME'] + '.zip';
with zipfile.ZipFile(zip, 'w') as zipf:
  zipf.write(os.environ['FW_FILE_NAME'])
  zipf.write('build.config')

In [ ]:
# @title Check firmware size
%%bash
partitions=padavan-ng/trunk/configs/boards/$CONFIG_VENDOR/$CONFIG_FIRMWARE_PRODUCT_ID/partitions.config
max_fw_size="$(awk '/Firmware/ { getline; getline; sub(",", ""); print strtonum($2); }' "$partitions")"
fw_size="$(stat -c %s "$FW_FILE_NAME")"
if ((fw_size > max_fw_size)); then
  fw_size_fmtd="$(numfmt --grouping "$fw_size") bytes"
  max_fw_size_fmtd="$(numfmt --grouping "$max_fw_size") bytes"
  echo "Firmware size ($fw_size_fmtd) exceeds max size ($max_fw_size_fmtd) for your target device"
  exit 1
fi

In [ ]:
# @title Download
zip = os.environ['CONFIG_VENDOR'] + '_' + os.environ['FW_NAME'] + '.zip';
if ZIP == 'download':
  files.download(zip)
else:
  ! mv "$zip" "/content/drive/MyDrive/"